## Working with Batches

The Batch API endpoint allows users to submit requests for asynchronous batch processing. We will process these requests within 24 hours. The details of each request will be read from a pre-uploaded file, and the responses will be written to an output file. You can query the batch object for status updates and results. Each model will be offered at 50% cost discount vs. the synchronous APIs. 



# Univeral Code Used for the Entire Notebook

Let's set up our libraries and client

In [24]:
# import libraries
from openai import OpenAI

In [25]:
# Create a client we can use
client = OpenAI()

## Preparing the Batch File

Batches start with a .jsonl file where each line contains the details of an individual request to the API. For now, the available endpoints are /v1/chat/completions (Chat Completions API) and /v1/embeddings (Embeddings API). For a given input file, the parameters in each line's body field are the same as the parameters for the underlying endpoint. Each request must include a unique custom_id value, which you can use to reference results after completion. Here's an example of an input file with 2 requests. Note that each input file can only include requests to a single model.

NOTE: For some insane reason you are required to indicate the API endpoint here and in the batch creation later on. Presumably, OpenAI will one day allow hitting multiple different APIs in one batch request; but today is not that day. 

<br/>
Examples:

```
{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Give me three paragraphs on the penguin lifecycle."}],"max_tokens": 1000}}

{"custom_id": "request-2", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Give me three paragraphs on penguin mating habits."}],"max_tokens": 1000}}

{"custom_id": "request-3", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Give me three paragraphs on penguin species differences."}],"max_tokens": 1000}}
```

### Uploading the File

After creating your batch file, you must upload it so that you can reference it correctly when kicking off batches. Upload your .jsonl file using the Files API.

In [26]:

bad_batch_input_file = client.files.create(
    file=open("./artifacts/badbatchinput.jsonl", "rb"),
    purpose="batch"
)

good_batch_input_file = client.files.create(
    file=open("./artifacts/goodbatchinput.jsonl", "rb"),
    purpose="batch"
)

## Creating the Batch

Once you've successfully uploaded your input file, you can use the input File object's ID to create a batch. For now, the completion window can only be set to 24h. You can also provide custom metadata via an optional metadata parameter.

In [31]:
bad_batch_input_file_id = bad_batch_input_file.id

bad_batch = client.batches.create(
    input_file_id=bad_batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "bad nightly penguin job"
    }
)

print(bad_batch)
print("\n\n")

good_batch_input_file_id = good_batch_input_file.id

good_batch = client.batches.create(
    input_file_id=good_batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "good nightly penguin job"
    }
)

print(good_batch)

Batch(id='batch_aeo2ZMmdavhEdeebQ5dNdCaz', completion_window='24h', created_at=1720318527, endpoint='/v1/chat/completions', input_file_id='file-AZy8R5vyhc4zsoG4Y6pVXSk1', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1720404927, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'bad nightly penguin job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))



Batch(id='batch_FtQXjIiZ1TQWtAEgPoYnXqwN', completion_window='24h', created_at=1720318527, endpoint='/v1/chat/completions', input_file_id='file-JwfDUxjAcYGRB7bYl2N8uCz8', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1720404927, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'good nightly penguin job'}, output_fil

## Checking Batch Status

The status of a given Batch object can be any of the following:

| STATUS      | DESCRIPTION                                                                  |
|-------------|------------------------------------------------------------------------------|
| validating  | the input file is being validated before the batch can begin                 |
| failed      | the input file has failed the validation process                             |
| in_progress | the input file was successfully validated and the batch is currently being run |
| finalizing  | the batch has completed and the results are being prepared                   |
| completed   | the batch has been completed and the results are ready                       |
| expired     | the batch was not able to be completed within the 24-hour time window        |
| cancelling  | the batch is being cancelled (may take up to 10 minutes)                     |
| cancelled   | the batch was cancelled                                                      |


In [28]:

print(client.batches.retrieve(bad_batch.id))
print("\n\n")
print(client.batches.retrieve(good_batch.id))

Batch(id='batch_uiGGOF2Ezj2ghrbyIVxkGOXZ', completion_window='24h', created_at=1720317806, endpoint='/v1/chat/completions', input_file_id='file-AZy8R5vyhc4zsoG4Y6pVXSk1', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='duplicate_custom_id', line=2, message='The custom_id for this request is a duplicate of another request. The custom_id parameter must be unique for each request in a batch.', param='custom_id'), BatchError(code='duplicate_custom_id', line=3, message='The custom_id for this request is a duplicate of another request. The custom_id parameter must be unique for each request in a batch.', param='custom_id')], object='list'), expired_at=None, expires_at=1720404206, failed_at=1720317807, finalizing_at=None, in_progress_at=None, metadata={'description': 'bad nightly penguin job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))



Batch(id='

## Listing the Batches

At any time, you can see all your batches. For users with many batches, you can use the limit and after parameters to paginate your results.

In [34]:
print(client.batches.list())

for batch in client.batches.list():
    print(batch.id)
    print(batch.status)
    print(batch.metadata.get("description"))
    print("\n\n")

SyncCursorPage[Batch](data=[Batch(id='batch_FtQXjIiZ1TQWtAEgPoYnXqwN', completion_window='24h', created_at=1720318527, endpoint='/v1/chat/completions', input_file_id='file-JwfDUxjAcYGRB7bYl2N8uCz8', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1720318538, error_file_id=None, errors=None, expired_at=None, expires_at=1720404927, failed_at=None, finalizing_at=1720318537, in_progress_at=1720318529, metadata={'description': 'good nightly penguin job'}, output_file_id='file-1yLXSMwL5oqaKUdkczfgmbBe', request_counts=BatchRequestCounts(completed=3, failed=0, total=3)), Batch(id='batch_aeo2ZMmdavhEdeebQ5dNdCaz', completion_window='24h', created_at=1720318527, endpoint='/v1/chat/completions', input_file_id='file-AZy8R5vyhc4zsoG4Y6pVXSk1', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='duplicate_custom_id', line=2, message='The custom_id for this request 